In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy

import sys
import os
import math

from pprint import pprint
import preprocess_utils as prep

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer

%matplotlib inline
%load_ext autoreload
%autoreload 2

is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
    
pd.set_option('display.max_columns', 500)

## [Example on Keras LSTM sentiment classification](https://github.com/explosion/spaCy/blob/master/examples/deep_learning_keras.py)

In [2]:
animes = prep.load_preprocessed_animes()

In [3]:
animes.head()

,score,synopsis,title,total,genres.is_Josei,genres.is_Supernatural,genres.is_Adventure,genres.is_Action,genres.is_Shoujo,genres.is_Ecchi,genres.is_Slice of Life,genres.is_Romance,genres.is_Harem,genres.is_Fantasy,genres.is_Sci-Fi,genres.is_Comedy,genres.is_Drama,genres.is_Seinen,genres.is_Mystery,genres.is_School,genres.is_Shounen,source.is_Light novel,source.is_Manga,source.is_Original,source.is_Visual novel,related.has_Prequel,related.has_Sequel,popularity.is_Very_Popular,popularity.is_Popular,popularity.is_Unpopular,popularity.is_is_Very_Unpopular,season.Fall,season.Spring,season.Summer,season.Winter,year.2005,year.2006,year.2007,year.2008,year.2009,year.2010,year.2011,year.2012,year.2013,year.2014,year.2015,year.2016,year.2017,year.2018,percentage.completed,percentage.favorites,percentage.dropped,percentage.scored_by,percentage.watching,percentage.plan_to_watch,percentage.on_hold,synopsis.vector_0,synopsis.vector_1,synopsis.vector_2,synopsis.vector_3,synopsis.vector_4,synopsis.vector_5,synopsis.vector_6,synopsis.vector_7,synopsis.vector_8,synopsis.vector_9,synopsis.vector_10,synopsis.vector_11,synopsis.vector_12,synopsis.vector_13,synopsis.vector_14,synopsis.vector_15,synopsis.vector_16,synopsis.vector_17,synopsis.vector_18,synopsis.vector_19,synopsis.vector_20,synopsis.vector_21,synopsis.vector_22,synopsis.vector_23,synopsis.vector_24,synopsis.vector_25,synopsis.vector_26,synopsis.vector_27,synopsis.vector_28,synopsis.vector_29,synopsis.vector_30,synopsis.vector_31,synopsis.vector_32,synopsis.vector_33,synopsis.vector_34,synopsis.vector_35,synopsis.vector_36,synopsis.vector_37,synopsis.vector_38,synopsis.vector_39,synopsis.vector_40,synopsis.vector_41,synopsis.vector_42,synopsis.vector_43,synopsis.vector_44,synopsis.vector_45,synopsis.vector_46,synopsis.vector_47,synopsis.vector_48,synopsis.vector_49,synopsis.vector_50,synopsis.vector_51,synopsis.vector_52,synopsis.vector_53,synopsis.vector_54,synopsis.vector_55,synopsis.vector_56,synopsis.vector_57,synopsis.vector_58,synopsis.vector_59,synopsis.vector_60,synopsis.vector_61,synopsis.vector_62,synopsis.vector_63,synopsis.vector_64,synopsis.vector_65,synopsis.vector_66,synopsis.vector_67,synopsis.vector_68,synopsis.vector_69,synopsis.vector_70,synopsis.vector_71,synopsis.vector_72,synopsis.vector_73,synopsis.vector_74,synopsis.vector_75,synopsis.vector_76,synopsis.vector_77,synopsis.vector_78,synopsis.vector_79,synopsis.vector_80,synopsis.vector_81,synopsis.vector_82,synopsis.vector_83,synopsis.vector_84,synopsis.vector_85,synopsis.vector_86,synopsis.vector_87,synopsis.vector_88,synopsis.vector_89,synopsis.vector_90,synopsis.vector_91,synopsis.vector_92,synopsis.vector_93,synopsis.vector_94,synopsis.vector_95,synopsis.vector_96,synopsis.vector_97,synopsis.vector_98,synopsis.vector_99,synopsis.vector_100,synopsis.vector_101,synopsis.vector_102,synopsis.vector_103,synopsis.vector_104,synopsis.vector_105,synopsis.vector_106,synopsis.vector_107,synopsis.vector_108,synopsis.vector_109,synopsis.vector_110,synopsis.vector_111,synopsis.vector_112,synopsis.vector_113,synopsis.vector_114,synopsis.vector_115,synopsis.vector_116,synopsis.vector_117,synopsis.vector_118,synopsis.vector_119,synopsis.vector_120,synopsis.vector_121,synopsis.vector_122,synopsis.vector_123,synopsis.vector_124,synopsis.vector_125,synopsis.vector_126,synopsis.vector_127,synopsis.vector_128,synopsis.vector_129,synopsis.vector_130,synopsis.vector_131,synopsis.vector_132,synopsis.vector_133,synopsis.vector_134,synopsis.vector_135,synopsis.vector_136,synopsis.vector_137,synopsis.vector_138,synopsis.vector_139,synopsis.vector_140,synopsis.vector_141,synopsis.vector_142,synopsis.vector_143,synopsis.vector_144,synopsis.vector_145,synopsis.vector_146,synopsis.vector_147,synopsis.vector_148,synopsis.vector_149,synopsis.vector_150,synopsis.vector_151,synopsis.vector_152,synopsis.vector_153,synopsis.vector_154,synopsis.vector_155,synopsis.vector_156,synopsis.vector_157,synopsis.vector_158,synopsis.vector_159,synopsis.vect

In [4]:
reviews = prep.get_reviews_from_anime()

In [5]:
reviews = prep.preprocess_reviews(reviews, use_review_embeddings=False)

In [6]:
reviews.head()

,title,mal_id,reviewer,content,helpful_count,date,reviewer.scores.animation,reviewer.scores.character,reviewer.scores.enjoyment,reviewer.scores.overall,reviewer.scores.sound,reviewer.scores.story,reviewer.percentage.episodes_seen
review_id,,,,,,,,,,,,,
12530,Eyeshield 21,15,Poparteeb,How long is it? 145 episodes. What’s it about?...,234,2009-02-18T09:29:00+00:00,8,8,9,8,7,8,1.000000
340,Eyeshield 21,15,VK11,Eyeshield 21 - Japan's take on American Footba...,84,2007-06-01T18:46:00+00:00,9,8,9,8,7,7,0.551724
2650,Eyeshield 21,15,kiddemon86,Eyeshield 21 is indeed an outstanding anime. F...,33,2008-02-23T00:06:00+00:00,10,10,10,10,8,8,0.510345
197794,Eyeshield 21,15,Pneumonic,**This review is spoiler free**\r\nAt first gl...,31,2015-09-04T07:19:00+00:00,8,10,7,7,6,6,1.000000
10554,Eyeshield 21,15,graysky14,"At first glance, many thought that American fo...",21,2008-12-21T23:19:00+00:00,8,10,10,10,10,9,1.000000


In [ ]:
reviews.to_hdf("data/bert_vectors/reviews.hdf5", "table")

In [19]:
nlp = spacy.load('en_pytt_bertbaseuncased_lg')

In [239]:
sample_1 = animes["synopsis"].iloc[256].replace("[Written by MAL Rewrite]", "")
doc_1 = nlp(sample_1)

sample_2 = animes["synopsis"].iloc[65].replace("[Written by MAL Rewrite]", "")
doc_2 = nlp(sample_2)

sample_3 = animes["synopsis"].iloc[120].replace("[Written by MAL Rewrite]", "")
doc_3 = nlp(sample_3)

doc_4 = nlp("Common side effects include trouble sleeping, sexual dysfunction, loss of appetite, dry mouth, rash, and abnormal dreams.[2] Serious side effects include serotonin syndrome, mania, seizures, an increased risk of suicidal behavior in people under 25 years old, and an increased risk of bleeding.[2] If stopped suddenly, a withdrawal syndrome may occur with anxiety, dizziness, and changes in sensation.[2] It is unclear if it is safe in pregnancy.[6] If already on the medication, it may be reasonable to continue during breastfeeding.[6] Its mechanism of action is not entirely clear but believed to be related to increasing serotonin activity in the brain")

doc_5 = nlp("Common side effects include sleepiness, depression, headaches, feeling tired, dry mouth, and memory problems.[3] Some of the sedation and tiredness may improve within a few days.[8] Due to concerns about misuse, some do not recommend alprazolam as an initial treatment for panic disorder.[9] Withdrawal or rebound symptoms may occur if use is suddenly decreased.[3] Other rare risks include suicide, possibly due to loss of inhibition.[10] Gradually decreasing the dose over weeks or months may be required.[6] Alprazolam, like other benzodiazepines, acts through the GABAA receptor.")

doc_6 = nlp("Year 198 of the Solar Era in Tokyo, special fire brigades are fighting against a phenomenon called spontaneous human combustion where humans beings are turned into living infernos called 'Infernals.' While the Infernals are first generation cases of spontaneous human combustion, later generations possess the ability to manipulate flames while retaining human form. Shinra Kusakabe, a youth who gained the nickname Devil's Footprints for his ability to ignite his feet at will, joins the Special Fire Force Company 8 which composes of other flames users as they work to extinguish any Infernals they encounter. As a faction that is creating Infernals appears, Shira begins to uncover the truth behind a mysterious fire that caused the death of his family twelve years ago.")

doc_7 = nlp("The Walking Dead, based on the comic book series written by Robert Kirkman and published by Image Comics, tells the story of the months and years that follow after a zombie apocalypse. It follows a group of survivors, led by police officer Rick Grimes, who travel in search of a safe and secure home. The comic goes on to explore the challenges of life in a world overrun by zombies who take a toll on the survivors, and sometimes the interpersonal conflicts present a greater danger to their continuing survival than the zombies that roam the country. Over time, the characters are changed by the constant exposure to death and some grow willing to do anything to survive.")

doc_8 = nlp("A struggling high school chemistry teacher, Walter White (Bryan Cranston), is diagnosed with inoperable, advanced lung cancer. On a ride along with his DEA agent brother-in-law Hank Schrader (Dean Norris), Walter sees a former student of his, Jesse Pinkman (Aaron Paul), fleeing the scene of a meth lab. He later contacts Jesse and devises a scheme to become partners in an attempt to combine their skills to manufacture and distribute methamphetamine, with Walter cooking the product and Jesse using his street connections to distribute it. Walter says he wants to provide financial stability for his pregnant wife, Skyler (Anna Gunn) and disabled son, and to pay for his expensive cancer treatment. During Walter and Jesse's first days of selling Albuquerque's finest meth, they encounter a series of problems with local drug dealers. He continues to produce meth despite these setbacks using the alias 'Heisenberg'.")

doc_9 = nlp("Walter continues to find himself facing insurmountable medical bills from his cancer treatment. Despite having had several bad experiences while producing meth with Jesse, Walter agrees to rejoin his partner. The two begin producing meth but run into multiple problems. Jesse's friend Badger (Matt L. Jones) is arrested while selling meth in a sting operation. Walter hires a lawyer, Saul Goodman (Bob Odenkirk), to help Badger. Walter and Jesse drive a recreational vehicle into the desert and produce meth for four days. Later, Combo, another of Jesse's friends and distributors, is killed by a rival gang for selling meth in their territory. Saul suggests the two find a new distribution model. Throughout this, Jesse has been building a relationship with his neighbor and landlord, Jane Margolis (Krysten Ritter). Jane, who is a recovering addict, relapses and the two begin doing heroin. Saul finds them a new business partner, Gus Fring (Giancarlo Esposito), who is willing to pay $1.2 million for the 38 pounds of meth they produced. Walter hastily delivers the product to Gus, but misses his daughter's birth. Walt withholds Jesse's half of the money because of his drug use, but Jane finds out about it and blackmails Walt. Walt visits Jesse's house and witnesses Jane overdosing and choking on her own vomit, but perceiving her as an enemy, sits idly beside her and lets her die. Skyler confronts Walter about his frequent absences and excuses. She begins to piece together his secret life and demands that they separate.")

doc_10 = nlp("When Kazuma Satou died, he was given two choices: pass on to heaven or be revived in a fantasy world. After choosing the new world, the goddess Aqua tasked him with defeating the Demon King, and let him choose any weapon to aid him. Unfortunately, Kazuma chose to bring Aqua herself and has regretted the decision ever since then. Not only is he stuck with a useless deity turned party archpriest, the pair also has to make enough money for living expenses. To add to their problems, their group continued to grow as more problematic adventurers joined their ranks. Their token spellcaster, Megumin, is an explosion magic specialist who can only cast one spell once per day and refuses to learn anything else. There is also their stalwart crusader, Lalatina 'Darkness Dustiness Ford, a helpless masochist who makes Kazuma look pure in comparison. Kono Subarashii Sekai ni Shukufuku wo! 2 continues to follow Kazuma and the rest of his party through countless more adventures as they struggle to earn money and have to deal with one another's problematic personalities. However, things rarely go as planned, and they are often sidetracked by their own idiotic tendencies.")

doc_11 = nlp("After dying a laughable and pathetic death on his way back from buying a game, high school student and recluse Kazuma Satou finds himself sitting before a beautiful but obnoxious goddess named Aqua. She provides the NEET with two options: continue on to heaven or reincarnate in every gamer's dream—a real fantasy world! Choosing to start a new life, Kazuma is quickly tasked with defeating a Demon King who is terrorizing villages. But before he goes, he can choose one item of any kind to aid him in his quest, and the future hero selects Aqua. But Kazuma has made a grave mistake—Aqua is completely useless! Unfortunately, their troubles don't end here; it turns out that living in such a world is far different from how it plays out in a game. Instead of going on a thrilling adventure, the duo must first work to pay for their living expenses. Indeed, their misfortunes have only just begun!")

doc_12 = nlp(reviews.iloc[0]["content"])

doc_13 = nlp(reviews.iloc[901]["content"])